In [64]:
import os
import numpy as np
import pandas as pd
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from scipy.ndimage import distance_transform_edt, label as scipy_label
from scipy.ndimage import binary_fill_holes
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.measure import regionprops
import xml.etree.ElementTree as ET
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# ==================== Configuration ====================
class Config:
    # Paths
    train_dir = "kaggle-data/train"
    test_dir = "kaggle-data/test_final"
    train_csv = "kaggle-data/train_ground_truth.csv"
    
    # Training
    img_size = 256
    batch_size = 8
    num_epochs = 50
    lr = 1e-4
    device = "cpu"
    num_workers = 0
    # Device - optimized for macOS + multiprocessing
    # if torch.backends.mps.is_available():
    #     device = "mps"  # Apple Silicon GPU
    #     num_workers = 0
    # elif torch.cuda.is_available():
    #     device = "cuda"  # NVIDIA GPU
    #     num_workers = 4
    # else:
    #     device = "cpu"   # CPU fallback
    #     num_workers = 2  # Reduced for CPU multiprocessing
    
    # Classes
    classes = ["Epithelial", "Lymphocyte", "Neutrophil", "Macrophage"]
    class_weights = [1.0, 1.0, 10.0, 10.0]  # For weighted loss
    num_classes = 4
    
    # Inference
    overlap = 0.5  # Sliding window overlap
    min_nucleus_size = 20  # Minimum pixels for a valid nucleus

config = Config()

# ==================== RLE Encoding/Decoding ====================
def rle_encode_instance_mask(mask: np.ndarray) -> str:
    """Convert instance mask to RLE string"""
    pixels = mask.flatten(order="F").astype(np.int32)
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    
    rle = []
    for i in range(0, len(runs)-1):
        start = runs[i]
        end = runs[i+1] if i+1 < len(runs) else len(pixels)-1
        length = end - start
        val = pixels[start]
        if val > 0:
            rle.extend([val, start, length])
    
    return "0" if not rle else " ".join(map(str, rle))

def rle_decode_instance_mask(rle: str, shape: tuple) -> np.ndarray:
    """Convert RLE string to instance mask"""
    if not rle or str(rle).strip() in ("", "0", "nan"):
        return np.zeros(shape, dtype=np.uint16)
    
    s = list(map(int, rle.split()))
    mask = np.zeros(shape[0]*shape[1], dtype=np.uint16)
    
    for i in range(0, len(s), 3):
        val, start, length = s[i], s[i+1], s[i+2]
        mask[start-1:start-1+length] = val
    
    return mask.reshape(shape, order="F")

# ==================== Data Loading ====================
def parse_xml_annotations(xml_path):
    """Parse XML annotations to get polygons per class"""
    tree = ET.parse(xml_path)
    root = tree.getroot()
    
    annotations = {cls: [] for cls in config.classes}
    
    for annotation in root.findall('.//Annotation'):
        class_name = annotation.get('Name')
        if class_name not in config.classes:
            continue
        
        for region in annotation.findall('.//Region'):
            vertices = []
            for vertex in region.findall('.//Vertex'):
                x = float(vertex.get('X'))
                y = float(vertex.get('Y'))
                vertices.append([x, y])
            
            if len(vertices) >= 3:
                annotations[class_name].append(np.array(vertices, dtype=np.int32))
    
    return annotations

def create_masks_from_annotations(annotations, img_shape):
    """Create instance and semantic masks from annotations"""
    h, w = img_shape[:2]
    
    # Instance masks per class
    instance_masks = {cls: np.zeros((h, w), dtype=np.uint16) for cls in config.classes}
    
    # Semantic mask (0=bg, 1-4=classes)
    semantic_mask = np.zeros((h, w), dtype=np.uint8)
    
    for cls_idx, cls_name in enumerate(config.classes, start=1):
        instance_id = 1
        for polygon in annotations[cls_name]:
            mask = np.zeros((h, w), dtype=np.uint8)
            cv2.fillPoly(mask, [polygon], 1)
            
            # Add to instance mask
            instance_masks[cls_name][mask > 0] = instance_id
            instance_id += 1
            
            # Add to semantic mask
            semantic_mask[mask > 0] = cls_idx
    
    return instance_masks, semantic_mask

# ==================== Dataset ====================
class CellDataset(Dataset):
    def __init__(self, df, img_dir, transform=None, is_train=True):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.is_train = is_train
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_id = row['image_id']
        
        # Load image
        img_path = os.path.join(self.img_dir, f"{img_id}.tif")
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.is_train:
            # Load masks from XML
            xml_path = os.path.join(self.img_dir, f"{img_id}.xml")
            annotations = parse_xml_annotations(xml_path)
            instance_masks, semantic_mask = create_masks_from_annotations(annotations, image.shape)
            
            # Create boundary map (for instance separation)
            boundary = self.create_boundary_map(semantic_mask, instance_masks)
            
            if self.transform:
                # Augment - ensure we resize all components
                transformed = self.transform(
                    image=image,
                    masks=[semantic_mask, boundary]
                )
                image = transformed['image']
                semantic_mask = transformed['masks'][0]
                boundary = transformed['masks'][1]
            else:
                # Apply resize even if no other transforms
                resize_transform = A.Resize(config.img_size, config.img_size)
                transformed = resize_transform(
                    image=image,
                    masks=[semantic_mask, boundary]
                )
                image = transformed['image']
                semantic_mask = transformed['masks'][0]
                boundary = transformed['masks'][1]
            
            # Convert to tensors
            image = torch.from_numpy(image).permute(2, 0, 1).float() / 255.0
            semantic_mask = torch.from_numpy(semantic_mask).long()
            boundary = torch.from_numpy(boundary).float().unsqueeze(0)
            
            return {
                'image': image,
                'semantic_mask': semantic_mask,
                'boundary': boundary,
                'img_id': img_id
            }
        else:
            # Test mode
            original_shape = image.shape[:2]
            
            # Always apply resize for test images
            if self.transform:
                transformed = self.transform(image=image)
                image = transformed['image']
            else:
                resize_transform = A.Resize(config.img_size, config.img_size)
                transformed = resize_transform(image=image)
                image = transformed['image']
            
            image_tensor = torch.from_numpy(image).permute(2, 0, 1).float() / 255.0
            
            return {
                'image': image_tensor,
                'img_id': img_id,
                'original_shape': original_shape
            }
    
    def create_boundary_map(self, semantic_mask, instance_masks):
        """Create boundary map for instance separation"""
        boundary = np.zeros_like(semantic_mask, dtype=np.float32)
        
        for cls_name in config.classes:
            inst_mask = instance_masks[cls_name]
            if inst_mask.max() == 0:
                continue
            
            # Find boundaries between instances
            for inst_id in range(1, inst_mask.max() + 1):
                mask = (inst_mask == inst_id).astype(np.uint8)
                if mask.sum() < 5:
                    continue
                
                # Erode to find boundary
                kernel = np.ones((3, 3), np.uint8)
                eroded = cv2.erode(mask, kernel, iterations=1)
                boundary_pixels = mask - eroded
                boundary[boundary_pixels > 0] = 1.0
        
        return boundary

# ==================== Model Architecture ====================
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        return x

class UNetWithBoundary(nn.Module):
    """U-Net style model with multi-task heads"""
    def __init__(self, num_classes=4):
        super().__init__()
        
        # Encoder
        self.enc1 = ConvBlock(3, 64)
        self.enc2 = ConvBlock(64, 128)
        self.enc3 = ConvBlock(128, 256)
        self.enc4 = ConvBlock(256, 512)
        
        self.pool = nn.MaxPool2d(2)
        
        # Bottleneck
        self.bottleneck = ConvBlock(512, 1024)
        
        # Decoder
        self.up4 = nn.ConvTranspose2d(1024, 512, 2, stride=2)
        self.dec4 = ConvBlock(1024, 512)
        
        self.up3 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.dec3 = ConvBlock(512, 256)
        
        self.up2 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.dec2 = ConvBlock(256, 128)
        
        self.up1 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.dec1 = ConvBlock(128, 64)
        
        # Multi-task heads
        self.semantic_head = nn.Conv2d(64, num_classes + 1, 1)  # +1 for background
        self.boundary_head = nn.Conv2d(64, 1, 1)
    
    def forward(self, x):
        # Encoder
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool(e1))
        e3 = self.enc3(self.pool(e2))
        e4 = self.enc4(self.pool(e3))
        
        # Bottleneck
        b = self.bottleneck(self.pool(e4))
        
        # Decoder with skip connections
        d4 = self.dec4(torch.cat([self.up4(b), e4], dim=1))
        d3 = self.dec3(torch.cat([self.up3(d4), e3], dim=1))
        d2 = self.dec2(torch.cat([self.up2(d3), e2], dim=1))
        d1 = self.dec1(torch.cat([self.up1(d2), e1], dim=1))
        
        # Multi-task outputs
        semantic = self.semantic_head(d1)
        boundary = torch.sigmoid(self.boundary_head(d1))
        
        return semantic, boundary

# ==================== Loss Functions ====================
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
    
    def forward(self, inputs, targets):
        # Ensure alpha is on the same device as inputs
        if self.alpha is not None:
            self.alpha = self.alpha.to(inputs.device)
        
        ce_loss = F.cross_entropy(inputs, targets, reduction='none', weight=self.alpha)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma * ce_loss).mean()
        return focal_loss

class DiceLoss(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, inputs, targets, smooth=1.0):
        inputs = F.softmax(inputs, dim=1)
        targets_one_hot = F.one_hot(targets, num_classes=inputs.shape[1]).permute(0, 3, 1, 2).float()
        
        intersection = (inputs * targets_one_hot).sum(dim=(2, 3))
        union = inputs.sum(dim=(2, 3)) + targets_one_hot.sum(dim=(2, 3))
        
        dice = (2. * intersection + smooth) / (union + smooth)
        return 1 - dice.mean()

class CombinedLoss(nn.Module):
    def __init__(self, class_weights, device):
        super().__init__()
        # Move weights to the specified device
        weights = torch.tensor(class_weights + [1.0], dtype=torch.float32).to(device)  # +1 for background
        self.focal = FocalLoss(alpha=weights, gamma=2.0)
        self.dice = DiceLoss()
        self.bce = nn.BCELoss()
    
    def forward(self, semantic_pred, boundary_pred, semantic_gt, boundary_gt):
        # Semantic loss
        semantic_loss = self.focal(semantic_pred, semantic_gt) + self.dice(semantic_pred, semantic_gt)
        
        # Boundary loss
        boundary_loss = self.bce(boundary_pred, boundary_gt)
        
        return semantic_loss + 0.5 * boundary_loss

# ==================== Training ====================
def get_transforms(is_train=True):
    if is_train:
        return A.Compose([
            A.Resize(config.img_size, config.img_size, always_apply=True),  # Always apply resize
            A.RandomRotate90(p=0.5),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=45, p=0.5),
            A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),
            A.GaussNoise(p=0.3),
            A.GaussianBlur(p=0.3),
        ])
    else:
        return A.Compose([
            A.Resize(config.img_size, config.img_size, always_apply=True),  # Always apply resize for val/test
        ])

def train_model():
   # Load data
    df = pd.read_csv(config.train_csv)
    
    # Split train/val
    from sklearn.model_selection import train_test_split
    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
    
    # Datasets - CRITICAL: both need transforms for resizing
    train_dataset = CellDataset(train_df, config.train_dir, transform=get_transforms(True), is_train=True)
    val_dataset = CellDataset(val_df, config.train_dir, transform=get_transforms(False), is_train=True)  # This line was missing transform!
    
    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False, num_workers=0)
    
    # Model
    model = UNetWithBoundary(num_classes=config.num_classes).to(config.device)
    
    # Loss and optimizer - pass device to criterion
    criterion = CombinedLoss(config.class_weights, config.device).to(config.device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.lr, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config.num_epochs)
    
    best_val_loss = float('inf')
    
    for epoch in range(config.num_epochs):
        # Training
        model.train()
        train_loss = 0
        
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{config.num_epochs} [Train]")
        for batch in pbar:
            images = batch['image'].to(config.device)
            semantic_gt = batch['semantic_mask'].to(config.device)
            boundary_gt = batch['boundary'].to(config.device)
            
            optimizer.zero_grad()
            
            semantic_pred, boundary_pred = model(images)
            loss = criterion(semantic_pred, boundary_pred, semantic_gt, boundary_gt)
            
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            pbar.set_postfix({'loss': loss.item()})
        
        train_loss /= len(train_loader)
        
        # Validation
        model.eval()
        val_loss = 0
        
        with torch.no_grad():
            for batch in val_loader:
                images = batch['image'].to(config.device)
                semantic_gt = batch['semantic_mask'].to(config.device)
                boundary_gt = batch['boundary'].to(config.device)
                
                semantic_pred, boundary_pred = model(images)
                loss = criterion(semantic_pred, boundary_pred, semantic_gt, boundary_gt)
                
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
        
        print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pth')
            print("🎉 New best model saved!")
        
        scheduler.step()
    
    return model

# ==================== Inference & Post-processing ====================
def watershed_instance_separation(semantic_pred, boundary_pred, class_id):
    """Separate touching nuclei using watershed"""
    # Get mask for this class
    class_mask = (semantic_pred == class_id).astype(np.uint8)
    
    if class_mask.sum() < config.min_nucleus_size:
        return np.zeros_like(class_mask, dtype=np.uint16)
    
    # Remove boundary regions
    class_mask = class_mask * (1 - boundary_pred)
    class_mask = (class_mask > 0.5).astype(np.uint8)
    
    # Distance transform
    dist = distance_transform_edt(class_mask)
    
    # Find local maxima as seeds
    coords = peak_local_max(dist, min_distance=5, labels=class_mask)
    
    if len(coords) == 0:
        return np.zeros_like(class_mask, dtype=np.uint16)
    
    # Create markers
    markers = np.zeros_like(class_mask, dtype=np.int32)
    for i, coord in enumerate(coords, start=1):
        markers[coord[0], coord[1]] = i
    
    # Watershed
    instance_mask = watershed(-dist, markers, mask=class_mask)
    
    # Remove small objects
    for region in regionprops(instance_mask):
        if region.area < config.min_nucleus_size:
            instance_mask[instance_mask == region.label] = 0
    
    # Relabel
    instance_mask = scipy_label(instance_mask > 0)[0]
    
    return instance_mask.astype(np.uint16)

def predict_image(model, image, original_shape):
    """Sliding window prediction with overlap"""
    h, w = original_shape
    patch_size = config.img_size
    stride = int(patch_size * (1 - config.overlap))
    
    # Initialize output
    semantic_output = np.zeros((h, w), dtype=np.float32)
    boundary_output = np.zeros((h, w), dtype=np.float32)
    count_map = np.zeros((h, w), dtype=np.float32)
    
    model.eval()
    with torch.no_grad():
        for y in range(0, h, stride):
            for x in range(0, w, stride):
                # Extract patch
                y1, y2 = y, min(y + patch_size, h)
                x1, x2 = x, min(x + patch_size, w)
                
                patch = image[:, y1:y2, x1:x2].unsqueeze(0).to(config.device)
                
                # Pad if needed
                if patch.shape[2] < patch_size or patch.shape[3] < patch_size:
                    patch = F.pad(patch, (0, patch_size - patch.shape[3], 0, patch_size - patch.shape[2]))
                
                # Predict
                semantic_pred, boundary_pred = model(patch)
                
                semantic_pred = F.softmax(semantic_pred, dim=1).cpu().numpy()[0]
                boundary_pred = boundary_pred.cpu().numpy()[0, 0]
                
                # Crop to original patch size
                semantic_pred = semantic_pred[:, :y2-y1, :x2-x1]
                boundary_pred = boundary_pred[:y2-y1, :x2-x1]
                
                # Accumulate
                semantic_class = np.argmax(semantic_pred, axis=0)
                semantic_output[y1:y2, x1:x2] += semantic_class
                boundary_output[y1:y2, x1:x2] += boundary_pred
                count_map[y1:y2, x1:x2] += 1
    
    # Average
    semantic_output = (semantic_output / (count_map + 1e-8)).astype(np.uint8)
    boundary_output = (boundary_output / (count_map + 1e-8))
    boundary_output = (boundary_output > 0.5).astype(np.uint8)
    
    return semantic_output, boundary_output

def generate_predictions(model, test_df):
    """Generate predictions for all test images"""
    predictions = []
    
    test_dataset = CellDataset(test_df, config.test_dir, transform=None, is_train=False)
    
    for i in tqdm(range(len(test_dataset)), desc="Predicting"):
        sample = test_dataset[i]
        image = sample['image']
        img_id = sample['img_id']
        original_shape = sample['original_shape']
        
        # Predict
        semantic_pred, boundary_pred = predict_image(model, image, original_shape)
        
        # Separate instances for each class
        instance_masks = {}
        for cls_idx, cls_name in enumerate(config.classes, start=1):
            instance_mask = watershed_instance_separation(semantic_pred, boundary_pred, cls_idx)
            instance_masks[cls_name] = instance_mask
        
        # Encode to RLE
        row = {'image_id': img_id}
        for cls_name in config.classes:
            rle = rle_encode_instance_mask(instance_masks[cls_name])
            row[cls_name] = rle
        
        predictions.append(row)
    
    return pd.DataFrame(predictions)

# ==================== Evaluation ====================
def calculate_iou(pred_mask, gt_mask):
    """Calculate IoU between two masks"""
    intersection = np.logical_and(pred_mask, gt_mask).sum()
    union = np.logical_or(pred_mask, gt_mask).sum()
    
    if union == 0:
        return 0.0
    return intersection / union

def calculate_wpq(pred_instance_masks, gt_instance_masks):
    """Calculate weighted Panoptic Quality"""
    class_pqs = []
    
    for cls_idx, cls_name in enumerate(config.classes):
        pred_mask = pred_instance_masks[cls_name]
        gt_mask = gt_instance_masks[cls_name]
        
        pred_ids = np.unique(pred_mask)[1:]  # Exclude background
        gt_ids = np.unique(gt_mask)[1:]
        
        if len(gt_ids) == 0 and len(pred_ids) == 0:
            pq = 1.0
        elif len(gt_ids) == 0:
            pq = 0.0
        elif len(pred_ids) == 0:
            pq = 0.0
        else:
            # Match predictions to ground truth
            tp = 0
            fp = 0
            fn = len(gt_ids)
            iou_sum = 0.0
            
            matched_gt = set()
            
            for pred_id in pred_ids:
                pred_nucleus = (pred_mask == pred_id)
                best_iou = 0.0
                best_gt_id = None
                
                for gt_id in gt_ids:
                    if gt_id in matched_gt:
                        continue
                    
                    gt_nucleus = (gt_mask == gt_id)
                    iou = calculate_iou(pred_nucleus, gt_nucleus)
                    
                    if iou > 0.5 and iou > best_iou:
                        best_iou = iou
                        best_gt_id = gt_id
                
                if best_gt_id is not None:
                    tp += 1
                    fn -= 1
                    iou_sum += best_iou
                    matched_gt.add(best_gt_id)
                else:
                    fp += 1
            
            if tp == 0:
                pq = 0.0
            else:
                sq = iou_sum / tp  # Segmentation Quality
                rq = tp / (tp + 0.5 * fp + 0.5 * fn)  # Recognition Quality
                pq = sq * rq
        
        class_pqs.append(pq)
    
    # Weighted average
    weights = config.class_weights
    wpq = sum(pq * w for pq, w in zip(class_pqs, weights)) / sum(weights)
    
    return wpq, class_pqs

def evaluate_on_validation(model, val_df):
    """Evaluate model on validation data with ground truth"""
    val_dataset = CellDataset(val_df, config.train_dir, transform=None, is_train=True)
    
    wpq_scores = []
    
    for i in tqdm(range(len(val_dataset)), desc="Evaluating Validation"):
        sample = val_dataset[i]
        image = sample['image']
        img_id = sample['img_id']
        original_shape = image.shape[1:]  # For validation, we use the transformed shape
        
        # Get ground truth from the dataset
        semantic_gt = sample['semantic_mask'].numpy()
        
        # Create ground truth instance masks from the dataset
        # For validation evaluation, we need to reconstruct instance masks from the dataset
        # Since we don't have the original XML during validation, we'll use a simplified approach
        # by treating each connected component in the semantic mask as an instance
        gt_instance_masks = {}
        for cls_idx, cls_name in enumerate(config.classes, start=1):
            class_mask = (semantic_gt == cls_idx).astype(np.uint8)
            if class_mask.sum() > 0:
                # Label connected components
                labeled_mask, num_features = scipy_label(class_mask)
                gt_instance_masks[cls_name] = labeled_mask.astype(np.uint16)
            else:
                gt_instance_masks[cls_name] = np.zeros_like(semantic_gt, dtype=np.uint16)
        
        # Predict using the model
        semantic_pred, boundary_pred = predict_image(model, image, original_shape)
        
        # Get predicted instances
        pred_instance_masks = {}
        for cls_idx, cls_name in enumerate(config.classes, start=1):
            instance_mask = watershed_instance_separation(semantic_pred, boundary_pred, cls_idx)
            pred_instance_masks[cls_name] = instance_mask
        
        # Calculate wPQ
        wpq, class_pqs = calculate_wpq(pred_instance_masks, gt_instance_masks)
        wpq_scores.append(wpq)
        
        print(f"{img_id}: wPQ = {wpq:.4f}, Class PQs = {[f'{pq:.4f}' for pq in class_pqs]}")
    
    mean_wpq = np.mean(wpq_scores)
    print(f"\n=== Mean Validation wPQ: {mean_wpq:.4f} ===\n")
    
    # Print detailed results
    for i, (img_id, wpq) in enumerate(zip(val_df['image_id'].values, wpq_scores)):
        print(f"{i+1:2d}. {img_id}: {wpq:.4f}")
    
    return mean_wpq

def evaluate_on_test_data(model, test_df, gt_df):
    """Evaluate model on test data with ground truth"""
    test_dataset = CellDataset(test_df, config.test_dir, transform=None, is_train=False)
    
    wpq_scores = []
    
    for i in tqdm(range(len(test_dataset)), desc="Evaluating"):
        sample = test_dataset[i]
        image = sample['image']
        img_id = sample['img_id']
        original_shape = sample['original_shape']
        
        # Predict
        semantic_pred, boundary_pred = predict_image(model, image, original_shape)
        
        # Get predicted instances
        pred_instance_masks = {}
        for cls_idx, cls_name in enumerate(config.classes, start=1):
            instance_mask = watershed_instance_separation(semantic_pred, boundary_pred, cls_idx)
            pred_instance_masks[cls_name] = instance_mask
        
        # Get ground truth instances
        gt_row = gt_df[gt_df['image_id'] == img_id].iloc[0]
        gt_instance_masks = {}
        for cls_name in config.classes:
            rle = gt_row[cls_name]
            gt_instance_masks[cls_name] = rle_decode_instance_mask(rle, original_shape)
        
        # Calculate wPQ
        wpq, class_pqs = calculate_wpq(pred_instance_masks, gt_instance_masks)
        wpq_scores.append(wpq)
        
        print(f"{img_id}: wPQ = {wpq:.4f}, Class PQs = {[f'{pq:.4f}' for pq in class_pqs]}")
    
    mean_wpq = np.mean(wpq_scores)
    print(f"\n=== Mean wPQ: {mean_wpq:.4f} ===")
    
    return mean_wpq

# ==================== Main ====================
if __name__ == "__main__":
    # Multiprocessing fix for macOS
    torch.multiprocessing.set_start_method('spawn', force=True)

    print("Starting Cell Segmentation & Classification Pipeline...")
    print(f"Device: {config.device}")
    
    # Train model
    print("\n=== Training Model ===")
    model = train_model()
    
    # Load best model
    model.load_state_dict(torch.load('best_model.pth'))
    
    # Generate test predictions
    print("\n=== Generating Test Predictions ===")
    test_files = [f.replace('.tif', '') for f in os.listdir(config.test_dir) if f.endswith('.tif')]
    test_df = pd.DataFrame({'image_id': test_files})
    
    predictions_df = generate_predictions(model, test_df)
    predictions_df.to_csv('submission.csv', index=False)
    print("Submission saved to submission.csv")
    
    # Evaluate on validation set with ground truth
    print("\n=== Evaluating on Validation Set ===")
    try:
        # Use validation set from training data
        df = pd.read_csv(config.train_csv)
        from sklearn.model_selection import train_test_split
        _, val_df = train_test_split(df, test_size=0.2, random_state=42)
        mean_wpq_val = evaluate_on_validation(model, val_df)
        # Evaluate on validation images
        mean_wpq = evaluate_on_test_data(model, val_df, test_df)
    except Exception as e:
        print(f"Evaluation error: {e}")
    
    print("\n=== Pipeline Complete ===")
    print("Check 'submission.csv' for final predictions")

Starting Cell Segmentation & Classification Pipeline...
Device: cpu

=== Training Model ===


Epoch 1/50 [Train]: 100%|██████████| 21/21 [02:04<00:00,  5.95s/it, loss=2.04]


Epoch 1 | Train Loss: 2.3098 | Val Loss: 2.1123
🎉 New best model saved!


Epoch 2/50 [Train]: 100%|██████████| 21/21 [02:27<00:00,  7.04s/it, loss=1.69]


Epoch 2 | Train Loss: 1.8091 | Val Loss: 1.6894
🎉 New best model saved!


...

slide197: wPQ = 1.0000, Class PQs = ['1.0000', '1.0000', '1.0000', '1.0000']


Evaluating Validation: 100%|██████████| 42/42 [00:38<00:00,  1.09it/s]


slide163: wPQ = 1.0000, Class PQs = ['1.0000', '1.0000', '1.0000', '1.0000']

=== Mean Validation wPQ: 1.0000 ===

 1. slide31: 1.0000
 2. slide172: 1.0000
 3. slide85: 1.0000
 4. slide199: 1.0000
 5. slide61: 1.0000
 6. slide156: 1.0000
 7. slide46: 1.0000
 8. slide182: 1.0000
 9. slide10: 1.0000
10. slide196: 1.0000
11. slide137: 1.0000
12. slide187: 1.0000
13. slide207: 1.0000
14. slide127: 1.0000
15. slide16: 1.0000
16. slide74: 1.0000
17. slide166: 1.0000
18. slide19: 1.0000
19. slide168: 1.0000
20. slide94: 1.0000
21. slide76: 1.0000
22. slide56: 1.0000
23. slide148: 1.0000
24. slide110: 1.0000
25. slide109: 1.0000
26. slide143: 1.0000
27. slide26: 1.0000
28. slide126: 1.0000
29. slide17: 1.0000
30. slide173: 1.0000
31. slide192: 1.0000
32. slide70: 1.0000
33. slide102: 1.0000
34. slide68: 1.0000
35. slide105: 1.0000
36. slide133: 1.0000
37. slide208: 1.0000
38. slide96: 1.0000
39. slide83: 1.0000
40. slide160: 1.0000
41. slide197: 1.0000
42. slide163: 1.0000


Evaluating:   0%|          | 0/42 [00:00<?, ?it/s]

Evaluation error: 'Epithelial'

=== Pipeline Complete ===
Check 'submission.csv' for final predictions
